# Visualize Windows smoke-test output

Creates manuscript-ready figures from `process_wsi_batch.py` smoke-test outputs.

In [ ]:
from pathlib import Path
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

# Point this to a smoke output folder, e.g. C:/path-rag-data/wsi_processed/smoke_YYYYMMDD_HHMMSS
SMOKE_OUTPUT_DIR = Path(r"C:/path-rag-data/wsi_processed/smoke_YYYYMMDD_HHMMSS")
FIGURE_DIR = SMOKE_OUTPUT_DIR / "figures"
FIGURE_DIR.mkdir(parents=True, exist_ok=True)

slide_dirs = sorted([p for p in SMOKE_OUTPUT_DIR.iterdir() if p.is_dir()])
if not slide_dirs:
    raise FileNotFoundError(f"No slide output folders found in {SMOKE_OUTPUT_DIR}")

slide_dir = slide_dirs[0]
print(f"Using slide folder: {slide_dir}")

In [ ]:
summary_path = slide_dir / "nuclei_summary.npz"
summary = np.load(summary_path)
nuclei_map = summary["nuclei_map"]
nuclei_count = int(summary["nuclei_count"])

patch_paths = sorted(slide_dir.glob("[0-9]*.png"), key=lambda p: int(p.stem))
if not patch_paths:
    raise FileNotFoundError(f"No ranked patch PNG files found in {slide_dir}")

plt.style.use("seaborn-v0_8-whitegrid")
fig, axes = plt.subplots(2, 3, figsize=(12, 8), constrained_layout=True)
for idx, ax in enumerate(axes.flat):
    if idx < len(patch_paths):
        image = Image.open(patch_paths[idx]).convert("RGB")
        ax.imshow(image)
        ax.set_title(f"Patch rank {idx + 1}", fontsize=11)
    ax.axis("off")

fig.suptitle(f"Top nuclei-rich patches (total nuclei: {nuclei_count})", fontsize=14, y=1.02)
patch_fig_path = FIGURE_DIR / "nuclei_top_patches.png"
fig.savefig(patch_fig_path, dpi=300, bbox_inches="tight")
plt.show()
print(f"Saved: {patch_fig_path}")

In [ ]:
fig, ax = plt.subplots(figsize=(6.5, 6), constrained_layout=True)
im = ax.imshow(nuclei_map, cmap="magma")
ax.set_title("Nuclei density map", fontsize=13)
ax.set_xlabel("x")
ax.set_ylabel("y")
cbar = fig.colorbar(im, ax=ax, fraction=0.046, pad=0.04)
cbar.set_label("Nuclei signal")

heatmap_fig_path = FIGURE_DIR / "nuclei_density_map.png"
fig.savefig(heatmap_fig_path, dpi=300, bbox_inches="tight")
plt.show()
print(f"Saved: {heatmap_fig_path}")